In [89]:
import pandas as pd
import json
import random
import numpy as np

In [40]:
projects = pd.read_csv("P3_GrantExport.csv", delimiter=';')

In [59]:
important = projects.iloc[:,[0,6,7,13]]
important.columns=["Number","Institution","University","Amount"]
important = important.set_index(["Number"])
important.head()

,Institution,University,Amount
Number,,,
1,NaN,Nicht zuteilbar - NA,11619.00
4,Faculté de Psychologie et des Sciences de l'Ed...,Université de Genève - GE,41022.00
5,Kommission für das Corpus philosophorum medii ...,"NPO (Biblioth., Museen, Verwalt.) - NPO",79732.00
6,Abt. Handschriften und Alte Drucke Bibliothek ...,Universität Basel - BS,52627.00
7,Schweiz. Thesauruskommission,"NPO (Biblioth., Museen, Verwalt.) - NPO",120042.00


In [67]:
with open('ch-cantons.topojson.json', encoding='utf-8') as data_file:
    json_file = json.loads(data_file.read())

In [77]:
objects=json_file['objects']
cantons=objects['cantons']['geometries']
cantons_df = pd.DataFrame(cantons)
cantons_df.head()

,arcs,id,properties,type
0,"[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]",ZH,{'name': 'Zürich'},Polygon
1,"[[[10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,...",BE,{'name': 'Bern/Berne'},MultiPolygon
2,"[[-12, 30, 31, 32, 33, 34]]",LU,{'name': 'Luzern'},Polygon
3,"[[35, 36, 37, 38, -16, 39, 40, 41]]",UR,{'name': 'Uri'},Polygon
4,"[[42, -33, 43, -1, 44, 45, 46, -42]]",SZ,{'name': 'Schwyz'},Polygon


In [80]:
cantons_ids=cantons_df.iloc[:,1]
cantons_ids

0     ZH
1     BE
2     LU
3     UR
4     SZ
5     OW
6     NW
7     GL
8     ZG
9     FR
10    SO
11    BS
12    BL
13    SH
14    AR
15    AI
16    SG
17    GR
18    AG
19    TG
20    TI
21    VD
22    VS
23    NE
24    GE
25    JU
Name: id, dtype: object

In [97]:
#Adding the canton column
size = len(important)
cantons_column = size*[""]
important["Canton"]=cantons_column
important.head()

,Institution,University,Amount,Canton
Number,,,,
1,NaN,Nicht zuteilbar - NA,11619.00,
4,Faculté de Psychologie et des Sciences de l'Ed...,Université de Genève - GE,41022.00,
5,Kommission für das Corpus philosophorum medii ...,"NPO (Biblioth., Museen, Verwalt.) - NPO",79732.00,
6,Abt. Handschriften und Alte Drucke Bibliothek ...,Universität Basel - BS,52627.00,
7,Schweiz. Thesauruskommission,"NPO (Biblioth., Museen, Verwalt.) - NPO",120042.00,


In [98]:
for i in important.index:
    random_int = random.randint(0,25)
    important.loc[i,"Canton"]=cantons_ids.loc[random_int]
important.head()

,Institution,University,Amount,Canton
Number,,,,
1,NaN,Nicht zuteilbar - NA,11619.00,TI
4,Faculté de Psychologie et des Sciences de l'Ed...,Université de Genève - GE,41022.00,OW
5,Kommission für das Corpus philosophorum medii ...,"NPO (Biblioth., Museen, Verwalt.) - NPO",79732.00,SH
6,Abt. Handschriften und Alte Drucke Bibliothek ...,Universität Basel - BS,52627.00,ZH
7,Schweiz. Thesauruskommission,"NPO (Biblioth., Museen, Verwalt.) - NPO",120042.00,FR


In [103]:
#important.iloc[:, 2] <- sapply(important.iloc[:, 2], as.numeric)
#amount_per_canton=important.groupby(['Canton'])['Amount'].sum()
#amount_per_canton

SyntaxError: invalid syntax (<ipython-input-103-f490e7de1299>, line 1)